### Format
    
    Input
    """
    질문 : <question>
    """
    
    Output
    """ 
    풀이 : <reasoning>
    정답 : <answer>
    

## 각각의 데이터는 같음
    - /userhomes/philhoon/kt-ai-challenge/result
    # kor-gsm8k-peft1-test.jsonl
    # kor-gsm8k-reason-test.jsonl

    # kor-gsm8k-peft1-train.csv
    # kor-gsm8k-reason-train.csv

In [4]:
import json
import requests
from src import utils
import time
import os
import csv
import re

In [15]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [19]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [20]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'
api = utils.read_json_file(api_path)

In [22]:
model = model_api(api['url'], api['headers'])

In [21]:
# input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
input_file = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft1-test.jsonl'

In [6]:
data = utils.read_jsonlines(input_file)

In [7]:
len(data)

1293

In [8]:
# def get_gsm8k_zs(que):
#     prompt = f'질문 : {que} \n정답 : '
#     return prompt

In [9]:
# def get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s):
#     prompt = f"질문: {kor_que}"
#     output = f"풀이: {kor_ans} 정답: {ans_s}"
#     return prompt, output

In [10]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [11]:
# from pprint import pprint

In [12]:
def remove_line_slit(context):
    context = re.sub(r"\n", " ", context)
    return context

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [13]:
from pprint import pprint

In [23]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    prompt = ins['kor_question_reason']

    print(prompt)
    body = get_body(prompt, BODY_TEMP)
    print(body)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
    break
    # checking whether there is an answer 
#     test = kor_que.split('\n')
    
#     if len(test[0]) != len(kor_que):
#         split_cnt += 1
#         continue
    
#     print(f'kor_que')
#     print(f'{kor_que}')
#     print(f'kor_ans')
#     print(f'{kor_ans}')
#     print(f'ans_s')
#     print(f'{ans_s}')
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s)
    
    
    input_lst.append(remove_line_slit(in_))
    output_lst.append(remove_line_slit(out_))

#     if cnt == 5:
#         break
#     else:
#         cnt += 1

질문: 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요?
{"serviceInstanceId": "ddm3kz9g", "nluType": "009", "apiType": "002", "utterance": "\uc9c8\ubb38: \ud55c \ubc8c\uc758 \ub85c\ube0c\ub97c \ub9cc\ub4dc\ub294 \ub370 \ud30c\ub780\uc0c9 \uc12c\uc720 2 \ub369\uc774\uc640 \uadf8 \uc808\ubc18\ub9cc\ud07c\uc758 \ud770\uc0c9 \uc12c\uc720\uac00 \ud544\uc694\ud569\ub2c8\ub2e4. \ucd1d \uba87 \ub369\uc774\uc758 \uc12c\uc720\uac00 \ud544\uc694\ud55c\uac00\uc694?"}


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyError: 'data'

In [ ]:
len(data) - split_cnt

In [ ]:
len(input_lst)

In [ ]:
len(output_lst)

In [ ]:
def save_csv_file(file_path, sentence_a_list, sentence_b_list):
    with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        for sentence_a, sentence_b in zip(sentence_a_list, sentence_b_list):
            writer.writerow([sentence_a, sentence_b])

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
output_file = output_path + '/kor-gsm8k-reason-train.csv'

In [ ]:
output_file

In [ ]:
save_csv_file(output_file, input_lst, output_lst)

In [ ]:
input_lst[1]

In [ ]:
output_lst[1]

### Making Test Set

In [ ]:
output_file = output_path + '/kor-gsm8k-reason-test.jsonl'

In [ ]:
print(output_file)

In [ ]:
jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
len(data)

In [ ]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    
    
    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s)
    
    ins['kor_question_reason'] = in_
    ins['kor_question_reason_answer'] = out_

    jsonl_writer.write_json_line(ins)

In [ ]:
len(data) - split_cnt

In [ ]:
split_cnt